In [17]:
from pymongo import MongoClient

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": "2023-04-12T08:00:00Z"},
        {"user_id": 1, "action": "view", "timestamp": "2023-04-12T09:00:00Z"},
        {"user_id": 2, "action": "purchase", "timestamp": "2023-04-12T10:00:00Z"}
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()


Data inserted successfully


## 문제 1
#### 데이터베이스에 새로운 필드로 genre 추가
#### 사용자는 fantasy 장르의 모든 책을 요청
#### 쿼리 작성 목표 : fantasy 장르에 해당하는 모든 책의 제목과 저자를 찾는 MongoDB 쿼리를 함수로 생성

In [ ]:
def find_genre(db, genre):
    # db 의 books 컬렉션을 선택하여 변수에 할당
    books_colloection = db.books

    # 검색 조건 : genre 필드가 입력된 장르와 일치하는 도큐먼트를 찾음
    query = {'genre': genre}

    # 검색 결과에 포함할 필드를 정의 : _id 는 제외하고 title, author 만 포함
    projection = {'_id': 0, 'title': 1, 'author': 1}

    # books_collection 에서 query 와 projection 조건에 맞는 도큐먼트를 검색
    books = books_colloection.find(query, projection)

    # 검색된 도큐먼트를 하나씩 반복하며 출력
    for book in books:
        print(book)

# Mongodb 클라이언트를 생성하여 데이터베이스에 연결
client = MongoClient('mongodb://localhost:27017/')

# local 데이터베이스를 선택하여 db 변수에 할당
db = client.local
find_genre(db, 'fantasy')   # 함수 실행
client.close()  # MongoDB 연결 종료

## 문제 2
#### 영화 감독 별로 영화 평점의 평균을 계산
#### 쿼리 작성 목표 : 모든 영화 감독의 영화 평점 평균을 계산하고, 평균이 높은 순으로 정렬하는 쿼리 함수 생성

In [ ]:
def calculate_avg_ratings(db):
    movies_collection = db.movies   # movie 컬렉션을 선택하여 변수에 할당
    
    # MongoDB aggregation 파이프라인 정의
    pipeline = [
        # 1. 감독별로 그룹화 하고 각 감독의 평균 rating 계산
        {'$group': {'_id': '$director', 'average_rating': {"$avg": "$rating"}}},
        # 2. average_rating 을 기준으로 내림차순 정렬
        {"$sort": {"average_rating": -1}}
    ]

    # pipeline 을 사용해 movie_collection 에서 aggregation 실행
    results = movies_collection.aggregate(pipeline)

    # aggregation 결과를 반복하며 출력
    for result in results:
        print(result)

client = MongoClient('mongodb://localhost:27017/')
db = client.local
calculate_avg_ratings(db)
client.close()

{'_id': 'Christopher Nolan', 'average_rating': 8.799999999999999}


## 문제 3
#### 특정 사용자의 최근 행동 로그 조회
#### 쿼리 작성 목표 : 사용자 ID가 1인 사용자의 최근 행동 5개를 최신순으로 정렬하여 조회하는 쿼리 함수 생성

In [ ]:
def find_recent_actions_by_user(db, user_id, limit = 5):
    user_actions_collection = db.user_actions   # user_action 컬렉션을 선택하여 변수에 할당
    query = {'user_id': user_id}    # user_id 필드가 함수 인자로 받은 user_id 와 일치하는 도큐먼트를 찾음
    sort_criteria = [('timestamp', -1)]     # 정렬 기준 : timestamp 필드를 기준으로 내림차순(-1) 정렬

    # 조건에 맞는 도큐먼트를 찾고(.find) 정렬한 뒤(.sort), 지정된 limit 수만큼 결과를 제한(.limit)
    actions = user_actions_collection.find(query).sort(sort_criteria).limit(limit)
    for action in actions:
        print(action)

client = MongoClient('mongodb://localhost:27017/')
db = client.local
find_recent_actions_by_user(db, 1)
client.close()

{'_id': ObjectId('672f7c30c4fbe6b6015944bf'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('672f7c30c4fbe6b6015944be'), 'user_id': 1, 'action': 'click', 'timestamp': '2023-04-12T08:00:00Z'}


## 문제 4
#### 데이터베이스에 저장된 책 데이터를 이용하여 각 출판 연도별로 출판된 책의 수를 계산
#### 쿼리 작성 목표 : 각 출판 연도별로 출판된 책의 수를 계산하고, 출판된 책의 수가 많은 순서대로 정렬하는 쿼리 함수 생성

In [ ]:
def count_by_year(db):
    books_collection = db.books     # books 컬렉션을 선택하여 변수에 할당

    pipeline = [
        # 년도별로 그룹화하고, 각 년도의 도큐먼트 수를 count 필드에 저장
        {"$group": {"_id": "$year", "count": {"$sum": 1}}},
        # 최신순으로 정렬
        {"$sort": {"count": -1}}
    ]

    results = books_collection.aggregate(pipeline)
    for result in results:
        print(result)

client = MongoClient('mongodb://localhost:27017/')
db = client.local
count_by_year(db)
client.close()

{'_id': 1987, 'count': 1}
{'_id': 2009, 'count': 1}
{'_id': 2002, 'count': 1}


## 문제 5
#### 특정 사용자의 행동 로그 중 특정 날짜 이전의 view 행동을 seen 으로 변경
#### 쿼리 작성 목표 : 사용자 ID가 1인 사용자의 23년 4월 10일 이전의 view 행동을 seen 으로 변경하는 업데이트 쿼리 함수 생성

In [ ]:
from datetime import datetime

def update_user_actions_before_date(db, user_id, date, old_action, new_action):
    user_actions_collection = db.user_actions   # user_actions 컬렉션을 선택 후 변수에 할당

    # user_id 필드가 지정된 user_id 와 action 필드가 지정된 old_action 과 일치하고
    # timestamp 필드가 지정된 date 보다 이전인 도큐먼트를 검색
    query = {'user_id': user_id, 'action': old_action, 'timestamp': {'$lt': date}}

    # 업데이트 내용 : action 의 필드에 새로운 값 new_action 으로 변경
    update = {'$set': {'action': new_action}}

    # update_many 함수를 사용해 조건에 부합한 모든 도큐먼트를 업데이트하고 결과 저장
    result = user_actions_collection.update_many(query, update)

    # 업데이트 된 도큐먼트의 개수를 출력
    print(f'Updated {result.modified_count} documents.')

client = MongoClient('mongodb://localhost:27017/')
db = client.local

# user_id = 1 이고 23년 4월 10일 이전, action 값이 view 인 경우 seen 으로 변경
update_user_actions_before_date(db, 1, datetime(2023, 4, 10), 'view', 'seen')
client.close()

Updated 0 documents.
